In [145]:
using Distributions
using Plots
using PGFPlotsX
using LaTeXStrings
using LinearAlgebra
using Colors
using JLD
using Parameters
using Random
pgfplotsx()
push!(PGFPlotsX.CUSTOM_PREAMBLE, raw"\usepgfplotslibrary{fillbetween}");

In [146]:
T = 80
n_steps = 50
gridman = LinRange(0, 1, n_steps);

### Heatmap

In [148]:
experiment = JLD.load("jlds/experiment.jld")

efe_grids = experiment["efe_grids"]
efe_grids = reshape(efe_grids, n_steps,n_steps, T);
efe_grids[efe_grids .== Inf] .= maximum(efe_grids[efe_grids .!= Inf])

epi_grids = experiment["epi_grids"]
epi_grids = reshape(epi_grids, n_steps,n_steps, T);

inst_grids = experiment["inst_grids"]
inst_grids = reshape(inst_grids, n_steps,n_steps, T);

idxs = experiment["idxs"]
points = experiment["points"];

In [149]:
index = 80
x = y = gridman

target = [[0.8], [0.2]]
current = map(x -> [first(x)], points[:,index])


axis1 = @pgf Axis(
    {
        title="EFE landscape at time $(index)",
        xlabel=L"g_s",
        ylabel=L"g_n",
        zlabel="hz",
        legend_pos = "north east",
        legend_cell_align="{left}",
        grid = "major",
        view = (0, 90),
        colorbar,
        "colormap/hot",
        colorbar_style={"xlabel=EFE, xticklabel pos=upper"}
    },
    Plot({only_marks, color="green", mark_size=3.0, mark="*"}, 
    Coordinates(target...)), LegendEntry("Target"),
    Plot({only_marks, color="black", mark_size=3.0, mark="*"}, 
    Coordinates(current...)), LegendEntry("Current"),
    Plot3(
        {
            surf,
            shader = "flat",
        },
        Coordinates(x, y, efe_grids[:, :, index]))
    )
pgfsave("tmp/heatmap_$(index).pdf", axis1)
pgfsave("tikz/heatmap_$(index).tikz", axis1)

### EFE plots

In [150]:
# Get the values of queried points for all time steps
epi_mins = [epi_grids[:,:,i][idxs[i]] for i in 1:T]
inst_mins = [inst_grids[:,:,i][idxs[i]] for i in 1:T]

# Get rid of infs for inhibition of return
epi_means = [mean(epi_grids[:,:,i][isfinite.(epi_grids[:,:,i])]) for i in 1:T]
inst_means = [mean(inst_grids[:,:,i][isfinite.(inst_grids[:,:,i])]) for i in 1:T];

In [151]:
plt_agent = @pgf GroupPlot(
    # group plot options
    {
        width="15cm", height="8cm",

        group_style = {
            group_size = "2 by 1",
            horizontal_sep = "1.5cm",
        },
    },
    # first
    {xlabel=L"t",
           ylabel="value",
           legend_pos = "north east",
           legend_cell_align="{left}",
           grid = "major",
        #    title="TODO: TITLE",
           legend_style = "{nodes={scale=0.8, transform shape}}",
    },
    Plot(
        {no_marks,color="blue"}, Coordinates(collect(1:T), epi_mins)
    ), LegendEntry("Epistemic value"),
    # second
    {xlabel=L"t",
           legend_pos = "south east",
           legend_cell_align="{left}",
           grid = "major",
        #    title="Dataset $(index)",
           legend_style = "{nodes={scale=0.8, transform shape}}",
    },
    Plot(
        {no_marks,color="blue"}, Coordinates(collect(1:T), inst_mins)
    ), LegendEntry("Instrumental value"),
)

pgfsave("tmp/grouped_agent.pdf", plt_agent)
pgfsave("tikz/grouped_agent.tikz", plt_agent)

In [152]:
# Plot of user pref function
include("../../src/environment/user.jl")
n_ticks = 100 # How finely to grid the preference function

x = y = LinRange(0,1,n_ticks)

z = zeros(n_ticks,n_ticks);
for i in 1:n_ticks
    for j in 1:100
	z[i,j] = generate_user_response([i/n_ticks,j/n_ticks],binary=false)
    end
end;

pref_map= @pgf Axis(
    {
        title="User preference function",
        xlabel=L"g_s",
        ylabel=L"g_n",
        zlabel="hz",
        legend_pos = "north east",
        legend_cell_align="{left}",
        grid = "major",
        view = (0, 90),
        colorbar,
        "colormap/hot",
        colorbar_style={"xlabel=θ, xticklabel pos=upper"}
    },
    Plot3(
        {
            surf,
            shader = "flat",
        },
        Coordinates(x, y, z))
    )
pgfsave("tmp/pref_map.pdf", pref_map)
pgfsave("tikz/pref_map.tikz", pref_map)

### Batch Plots

In [157]:
using Colors
experiment_1 = JLD.load("jlds/batch_experiment_1.jld")
experiment_2 = JLD.load("jlds/batch_experiment_2.jld")

# Heatmap of positive user appraisals
responses_1 = experiment_1["responses"]
responses_2 = experiment_2["responses"]
resp = vcat(responses_1,responses_2);

# Histogram of first index of positive response
idxs = [findfirst( isequal(1),resp[i,:]) for i in 1:size(resp)[1]]
idxs[isnothing.(idxs)] .= 82 # If the agent didn't get a thumbsup, set it to aft;

In [158]:
y = 1:T 
x = 1:T+1

resp_map= @pgf Axis(
    {
        title="Positive Appraisals",
        xlabel="Time Index",
        ylabel="Agent number",
        grid = "minor",
        view = (0, 90), 
        colormap = "{blackwhite}{color=(black) color=(yellow)}"
    },
    Plot3(
        {
            surf,
            shader = "flat corner",
        },
        Coordinates(x, y, resp')),
    )
pgfsave("tmp/resp_map.pdf", resp_map)
pgfsave("tikz/resp_map.tikz", resp_map)

In [166]:
using StatsBase: Histogram, fit, normalize
h = fit(Histogram, Float64.(idxs), nbins=100, closed = :left)

shift = 2
plt_histogram = @pgf Axis(
    {
        xlabel="\\small{Time Index}",
        ylabel="\\small{Number of Postive responses}",
        title="Ensemble Performance",
        ticklabel_style={"font=\\tiny"},
        yticklabel_style={
        "/pgf/number format/fixed,
        /pgf/number format/precision=0"
        },
        grid="major",
        yminorgrids=true,
        tick_align="outside",
        xtick=range(0, 80; length=5),
        ytick=range(1, 18; length=9),
        xmin =0.0,
        xmax=maximum(idxs)+shift,
        scaled_y_ticks = false,
    },
    Plot({"ybar interval", fill="blue!85", "forget plot"}, Table(h)))
pgfsave("tikz/ensemble_perf.tikz", plt_histogram)
pgfsave("tmp/ensemble_perf.pdf", plt_histogram)